In [1]:
# ! pip install opencv-python
import os
import copy
import numpy as np
import pickle as pkl
import pandas as pd
from collections import defaultdict
from datetime import datetime
import cv2
def parse_and_fuse_timestamp(time_str, time_nano):
    # Parse the time string (e.g., "2024-04-17 09:19:41") and timeNano (e.g., "322223")
    dt = datetime.strptime(f"{time_str}_{time_nano.split('_')[-1]}", "%Y-%m-%d_%H_%M_%S_%f")
    return dt

def process_pkl_files(meta_dir_root_dir, output_dir, background_image_name):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Dictionary to store tracks, where each key is a Label and value is a list of detections
    tracks = defaultdict(list)

    # create a frame with the background image
    background_image = cv2.imread(background_image_name)
    # Iterate through all .pkl files in the Meta directory
    meta_dir = os.path.join(meta_dir_root_dir, 'Meta')
    list_of_pkl_files = os.listdir(meta_dir)
    image_dir = os.path.join(meta_dir_root_dir, 'Images')
    list_of_jpg_files = os.listdir(image_dir)
    # max_x = 0
    # max_y = 0
    for pkl_indx, pkl_file in enumerate(list_of_pkl_files):
        # background_image_copy = copy.deepcopy(background_image)
        if pkl_file.endswith('.pkl') :
            # frame_bbox_image = cv2.imread(os.path.join(image_dir, list_of_jpg_files[pkl_indx]))
            pkl_path = os.path.join(meta_dir, pkl_file)
            # Read the .pkl file
            with open(pkl_path, 'rb') as f:
                data = pkl.load(f)

            # Extract required fields
            label = data['Label']
            if label == -10:
                continue
            time_nano = data['timeNano']
            time = data['time']
            box = data['BOX']

            # Fuse time and timeNano for sorting
            fused_timestamp = parse_and_fuse_timestamp(time, time_nano)

            # change fused_timestamp to a counter in a resolution of 1/30 seconds from 1/1/1900
            fused_timestamp = int((fused_timestamp - datetime(2023, 2, 9)).total_seconds() * 30)

            # Convert BOX [x1, y1, x2, y2] to four points [x1, y1, x2, y1, x2, y2, x1, y2]
            y1, x1, y2, x2 = [int(x) for x in box]
            box_points = [x1, y1, x2, y1, x1, y2, x2, y2]
            # if max_x < x2:
            #     max_x = x2
            # if max_y < y2:
            #     max_y = y2
            # #put the frame_bbox_image inside backjground_image at location of box_points
            # background_image_copy[x1:x2, y1:y2] = frame_bbox_image
            # cv2.imwrite(os.path.join(output_dir, background_image_name), background_image_copy)

            # print(label)
            # Append detection to the track for this Label
            tracks[label].append({
                'timestamp': fused_timestamp,
                'box_points': box_points,
                'source_file_name':pkl_file.split('.pkl')[0]
                # 'background_image': background_image_copy
            })
            print(f'\rprocessing pkl #{pkl_indx}/{len(list_of_pkl_files)}', end="")

    # print(f'max_y={max_y}')
    # print(f'max_x = {max_x}')
    # Process each track (Label) and save its detections to a CSV file
    indx = 0
    for label, detections in tracks.items():
        indx+=1
        print(f'\rprocessing track #{indx}/{len(list_of_pkl_files)}', end="")

        # Sort detections by timestamp to ensure chronological order within the track
        detections = sorted(detections, key=lambda x: x['timestamp'])

        # Prepare data for CSV: each row is a detection in the track
        csv_data = []
        for i, detection in enumerate(detections, 0):  # 0-based index for detections
            box_points = detection['box_points']
            # Pad box points with zeros to reach 32 values
            padded_points = box_points + [0.0] * (33 - len(box_points))
            # Create row: detection index + box points + final zero
            row = [detection['timestamp']] + padded_points + [0.0] + [detection['source_file_name']]
            csv_data.append(row)

        # Create a DataFrame without headers for the track
        df = pd.DataFrame(csv_data)

        # Define CSV filename (e.g., track_001.csv)
        csv_filename = f"1_{str(label).zfill(3)}.csv"
        csv_path = os.path.join(output_dir, csv_filename)

        # Save the track's detections to CSV without headers
        df.to_csv(csv_path, index=False, header=False)
        print(f"Saved track for label {label} to {csv_path} with {len(detections)} detections")

def main():
    string_date = '2023_2_10'
    # training_or_testing_str = 'training'
    training_or_testing_str = 'testing'
    # Define paths
    meta_dir = '/home/pp/Desktop/datasets/elsec_dataset/frame/01/' + string_date + '/Pos'  # Path to Meta directory
    output_dir = '/home/pp/Desktop/datasets/trajrec_data/elsec_data/'+ training_or_testing_str+'/trajectories/' + string_date  # Path to save CSV files
    background_image_name = '/home/pp/Desktop/datasets/elsec_dataset/frame/01/elsec_road_meshulash_times_check.jpg'
    # Process the .pkl files and save tracks
    process_pkl_files(meta_dir, output_dir, background_image_name)

if __name__ == '__main__':
    main()

processing track #1/438506Saved track for label 123516 to /home/pp/Desktop/datasets/trajrec_data/elsec_data/testing/trajectories/2023_2_10/1_123516.csv with 737 detections
processing track #2/438506Saved track for label 133045 to /home/pp/Desktop/datasets/trajrec_data/elsec_data/testing/trajectories/2023_2_10/1_133045.csv with 41 detections
processing track #3/438506Saved track for label 135809 to /home/pp/Desktop/datasets/trajrec_data/elsec_data/testing/trajectories/2023_2_10/1_135809.csv with 138 detections
processing track #4/438506Saved track for label 134269 to /home/pp/Desktop/datasets/trajrec_data/elsec_data/testing/trajectories/2023_2_10/1_134269.csv with 370 detections
processing track #5/438506Saved track for label 120059 to /home/pp/Desktop/datasets/trajrec_data/elsec_data/testing/trajectories/2023_2_10/1_120059.csv with 32 detections
processing track #6/438506Saved track for label 137142 to /home/pp/Desktop/datasets/trajrec_data/elsec_data/testing/trajectories/2023_2_10/1_1

In [2]:
import pandas as pd
import numpy as np
# Read the label data
label_df = pd.read_csv('/home/pp/Desktop/datasets/trajrec_data/elsec_data/testing/trajectories/2023_2_10/116915.csv',
                       names=['time', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4'],
                       delimiter='\t')

# Read anomaly data
anomaly_df = pd.read_csv('/home/pp/Desktop/datasets/elsec_dataset/9-10_02_2023_elsec_road_meshulash/anomaly.csv')

# Extract label from index column 
anomaly_df['label'] = anomaly_df['index'].str.extract(r'_(\d+)_')

# Print the loaded dataframes
# print("Label data:")
# # print(label_df)
# print("\nAnomaly data:")
# print(anomaly_df)


In [3]:
# Get list of trajectory files
import os
import pandas as pd

trajectory_path = '/home/pp/Desktop/datasets/trajrec_data/elsec_data/testing/trajectories/2023_2_10/'
trajectory_files = os.listdir(trajectory_path)

# Initialize empty list to store all times 
all_times = []

# Read each trajectory file and collect times
for indx, file in enumerate(trajectory_files):
    print(f"\rProcessing trajectory {indx + 1}/{len(trajectory_files)}", end="")
    if file.endswith('.csv') and file.split('.csv')[0] !='-10':
            df = pd.read_csv(os.path.join(trajectory_path, file),
                             names=['time'] + [f'col{i}' for i in range(1, 36)],
                             delimiter=',', index_col=False)

            trajectory_id = file.split('.')[0]
            times = df['time'].tolist()

            # Check if this trajectory is in anomaly_df
            is_anomaly = int(trajectory_id in anomaly_df['label'].values)

            # Add times with their anomaly status
            all_times.extend([(t, is_anomaly) for t in times])

# Create DataFrame from collected times and check for duplicates
result_df = pd.DataFrame(all_times, columns=['frame_time', 'is_anomaly'])

# Sort values and check duplicates  
result_df = result_df.sort_values('frame_time')
duplicates = result_df[result_df.duplicated(subset=['frame_time'], keep=False)]
# print("Duplicate frame times:")
# print(duplicates)

# Remove duplicates using boolean OR operation
result_df = result_df.groupby('frame_time')['is_anomaly'].agg(lambda x: x.max() if any(x == 1) else 0).reset_index()

# Display final sorted result
print("\nFinal sorted result without duplicates:")
print(result_df)


Processing trajectory 30542/30542
Final sorted result without duplicates:
        frame_time  is_anomaly
0                0           0
1                1           0
2                2           0
3                3           0
4                4           0
...            ...         ...
715555     5180996           0
715556     5180997           0
715557     5180999           0
715558     5181000           0
715559     5181002           0

[715560 rows x 2 columns]


In [4]:
result_df.to_csv('/home/pp/Desktop/datasets/trajrec_data/elsec_data/testing/frame_level_masks/2023_2_10/result.csv', index=False)

In [5]:
import os
import pandas as pd
result_df = pd.read_csv('/home/pp/Desktop/datasets/trajrec_data/elsec_data/testing/frame_level_masks/2023_2_10/result.csv')
result_df.set_index('frame_time', inplace=True)

trajectory_path = '/home/pp/Desktop/datasets/trajrec_data/elsec_data/testing/trajectories/2023_2_10/'
trajectory_files = os.listdir(trajectory_path)

for inde , file in enumerate(trajectory_files):
    if file.endswith('.csv') and file != 'result.csv' and file.split('.csv')[0] != '-10':
        print(f'{file} {inde} out of {len(trajectory_files)}')
        # Read original CSV
        df = pd.read_csv(os.path.join(trajectory_path, file),
                         names=['time'] + [f'col{i}' for i in range(1, 36)],
                         delimiter=',', index_col=False)

        # Get the frame time values from result_df
        frame_time_map = {time: pos for time, pos in zip(df['time'], result_df.index.get_indexer(df['time']))}

        # Update the time column
        df['time'] = df['time'].map(frame_time_map)

        df = df.drop(df.columns[-1], axis=1)

        # Write back to the same file
        df.to_csv(os.path.join(trajectory_path, file), index=False, header=False)


1_137093.csv 0 out of 30542
138736.csv 1 out of 30542
1_140284.csv 2 out of 30542
1_130563.csv 3 out of 30542
1_138256.csv 4 out of 30542
139344.csv 5 out of 30542
1_119789.csv 6 out of 30542
1_137223.csv 7 out of 30542
134758.csv 8 out of 30542
1_117453.csv 9 out of 30542
122252.csv 10 out of 30542
1_138788.csv 11 out of 30542
121710.csv 12 out of 30542
1_125734.csv 13 out of 30542
133360.csv 14 out of 30542
118628.csv 15 out of 30542
1_121906.csv 16 out of 30542
1_120132.csv 17 out of 30542
124721.csv 18 out of 30542
118493.csv 19 out of 30542
137109.csv 20 out of 30542
1_120669.csv 21 out of 30542
1_135151.csv 22 out of 30542
1_138734.csv 23 out of 30542
1_127189.csv 24 out of 30542
138025.csv 25 out of 30542
1_116974.csv 26 out of 30542
1_135327.csv 27 out of 30542
119344.csv 28 out of 30542
1_122555.csv 29 out of 30542
1_120720.csv 30 out of 30542
1_133809.csv 31 out of 30542
1_134028.csv 32 out of 30542
131904.csv 33 out of 30542
138405.csv 34 out of 30542
1_120666.csv 35 out of 

In [8]:
np.save('/home/pp/Desktop/datasets/trajrec_data/elsec_data/testing/frame_level_masks/2023_2_10/2023-2-10_1.npy',np.array(result_df.values).reshape(-1))
